<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_10_3_transformer_timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-558: Applications of Deep Neural Networks
**Module 10: Time Series in PyTorch**  

* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

* Part 10.1: Time Series Data Encoding for Deep Learning, PyTorch [[Video]](https://www.youtube.com/watch?v=CZi5Avp6p1s&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_1_timeseries.ipynb)
* Part 10.2: LSTM-Based Time Series with PyTorch [[Video]](https://www.youtube.com/watch?v=hIQLy5zCgH4&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_2_lstm.ipynb)
* **Part 10.3: Transformer-Based Time Series with PyTorch** [[Video]](https://www.youtube.com/watch?v=NGzQpphf_Vc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_3_transformer_timeseries.ipynb)
* Part 10.4: Seasonality and Trend [[Video]](https://www.youtube.com/watch?v=HOkxoLaUF9s&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_4_seasonal.ipynb)
* Part 10.5: Predicting with Meta Prophet [[Video]](https://www.youtube.com/watch?v=MzjMVsz0GyA&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_10_5_prophet.ipynb)

# Google CoLab Instructions

The following code checks that Google CoLab is and sets up the correct hardware settings for PyTorch.


In [1]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: not using Google CoLab


ModuleNotFoundError: No module named 'torch'

# Part 10.5: Transformers for TimeSeries in PyTorch

The transformative landscape of deep learning has witnessed monumental strides in the recent past, particularly in the domain of Natural Language Processing (NLP). Central to this revolution has been the advent of transformer architectures, which, with their attention mechanisms, have pushed the boundaries of what's achievable in tasks like machine translation, sentiment analysis, and language modeling. However, while transformers initially rose to prominence primarily within the realm of NLP, their applicability isn't restricted to just textual data. A growing wave of interest has emerged around leveraging these models for time-series predictions—a challenge that, though numerically distinct, bears conceptual resemblance to understanding sequences in language.

In time-series prediction, the objective often centers around forecasting future values based on historical data. This could involve predicting stock prices, weather patterns, or even the consumption of electricity in a region. At its core, this is a sequence-to-sequence task, where the past values form an input sequence and the future values to be predicted form an output sequence. Now, consider the similarities with machine translation in NLP, where an input sequence (sentence) in one language is translated into an output sequence in another language. Both scenarios require the model to recognize patterns, interdependencies, and context across sequences.

This chapter delves deep into the nuances of using PyTorch transformers for time-series prediction. We will embark on this journey by first establishing a foundational understanding of how transformers operate within the NLP space, before segueing into their adaptation for numeric sequences. By juxtaposing these two applications, readers will gain a comprehensive appreciation of the transformer's versatility and the subtle considerations required when transitioning from text to time.



## Loading Sun Spot Data for a Transformer Time Series (encoder only)

We will look at the same sunspot data as the previous section. However, this time we will use a transformer to predict. You can find the data files needed for this example at the following location.

* [Sunspot Data Files](http://www.sidc.be/silso/datafiles#total)
* [Download Daily Sunspots](http://www.sidc.be/silso/INFO/sndtotcsv.php) - 1/1/1818 to now.

We use the following code to load the sunspot file:


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

names = ['year', 'month', 'day', 'dec_year', 'sn_value',
         'sn_error', 'obs_num', 'unused1']
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/SN_d_tot_V2.0.csv",
    sep=';', header=None, names=names,
    na_values=['-1'], index_col=False)

print(df.head())
df.info()

ModuleNotFoundError: No module named 'numpy'

The data preprocessing is the same as was introduced in the previous section. We will use data before the year 2000 as training, the rest is used for validation.

In [3]:
# Data Preprocessing
print(df[df['obs_num'] == 0].index.tolist())
print(max(df[df['obs_num'] == 0].index.tolist()))
start_id = max(df[df['obs_num'] == 0].index.tolist()) + 1
df = df[start_id:].copy() #start from obs_num > 0
df['sn_value'] = df['sn_value'].astype(float)
df_train = df[df['year'] < 2000] #split time series into test and train
df_test = df[df['year'] >= 2000]

spots_train = df_train['sn_value'].to_numpy().reshape(-1, 1) #select sn_value column and reshape
spots_test = df_test['sn_value'].to_numpy().reshape(-1, 1)
print('spots_train:', spots_train)


scaler = StandardScaler() #rescale z = (x - mean) / std --> mean 0 std of 1
spots_train = scaler.fit_transform(spots_train).flatten().tolist()
spots_test = scaler.transform(spots_test).flatten().tolist()

print(spots_train)

[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 19, 20, 21, 22, 23, 25, 26, 29, 30, 31, 32, 34, 37, 38, 40, 41, 42, 47, 48, 50, 52, 54, 58, 59, 60, 61, 62, 63, 65, 66, 67, 69, 70, 71, 75, 80, 83, 84, 88, 89, 90, 91, 92, 96, 106, 109, 110, 111, 112, 123, 126, 135, 136, 137, 140, 161, 162, 163, 167, 175, 186, 192, 193, 196, 199, 201, 202, 205, 209, 211, 212, 222, 225, 226, 228, 231, 232, 235, 237, 238, 239, 247, 248, 253, 254, 259, 260, 261, 263, 264, 269, 270, 272, 277, 283, 284, 287, 288, 295, 298, 302, 303, 304, 305, 307, 309, 310, 311, 313, 314, 315, 316, 322, 326, 327, 329, 330, 331, 332, 333, 334, 335, 340, 341, 342, 343, 346, 347, 349, 351, 353, 354, 355, 358, 359, 361, 362, 363, 366, 370, 372, 373, 374, 375, 377, 378, 381, 383, 384, 385, 386, 388, 389, 390, 391, 394, 395, 397, 398, 399, 402, 403, 404, 406, 407, 409, 411, 413, 414, 416, 417, 418, 420, 422, 423, 424, 426, 429, 447, 448, 450, 451, 453, 456, 457, 474, 477, 478, 479, 480, 522, 527, 532, 534, 538, 539, 545, 546, 547, 5

Just like we did for LSTM in the previous section, we again break the data into sequences.

In [4]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)] # take a window starting at i with size seq len (input x)
        after_window = obs[i + seq_size] # value after window (predicted val y)
        x.append(window)
        y.append(after_window)

    print('x',len(x))
    print('y',len(y))
    return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

x_train, y_train = to_sequences(SEQUENCE_SIZE, spots_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, spots_test)

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) #batches for parallel training
# training in batches, which helps in both computational efficiency and generalization.
# Batched training ensures that the model updates its weights based on the average gradient over several data points

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


x 55150
y 55150
x 6381
y 6381


# Position Encoding for Transformers

In the realm of the transformer architecture, a pivotal component that ensures the model's success is its ability to consider the sequence's order. Unlike traditional RNNs or LSTMs, which process sequences step-by-step and inherently respect their order, transformers process all tokens in a sequence simultaneously. While this parallel processing significantly boosts computational efficiency and allows for long-range dependencies to be captured more effectively, it also means that transformers, in their native form, are oblivious to the position or order of tokens in a sequence. This is where the concept of positional encoding comes into play.

Positional encoding is a mechanism to provide the transformer with information about the position of tokens within a sequence. Essentially, it infuses order information into the otherwise position-agnostic embeddings. By adding positional encodings to the token embeddings before feeding them into the transformer, each token's position in the sequence becomes discernible to the model.

Positional encodings are vectors that get added to the embeddings of tokens. The intuition is to design these vectors in such a way that their values or patterns are unique for each position, allowing the model to differentiate between different positions in the sequence.

A popular method to generate positional encodings is using sinusoidal functions. For a given position $p$ in the sequence and dimension $d$ of the embedding, the positional encoding is computed as:

$$ PE(2,i) = \sin(\frac{p}{10000^{2i/d}}) $$
$$ PE(2,i+1) = \cos(\frac{p}{10000^{2i/d}}) $$

Where $i$ is the dimension index. These sinusoidal functions generate values between -1 and 1 and ensure a unique and repeatable pattern for each position.

The choice of sinusoidal functions isn't arbitrary. They have two compelling properties:

1. They produce values between -1 and 1, making them compatible with most embedding value ranges.
2. Their patterns allow the model to extrapolate positions beyond the sequence lengths seen during training.

One might wonder, why not just append or add the token's position as an integer to the embedding? The challenge with this approach is scale. Embedding values, especially after being trained, can exist within a specific range, and directly adding large integers (for tokens further down in long sequences) might disrupt the information in the original embeddings.

Furthermore, using raw integers wouldn't provide a consistent way for the model to generalize or extrapolate to sequence lengths not seen during training. Sinusoidal functions, in contrast, offer a predictable pattern that aids in such extrapolation.

The following code describes a simple implementation of a transformer-based model using PyTorch's built-in functionalities. The **TransformerModel** class encapsulates a transformer-based neural network designed for sequence processing. Upon initialization, the model sets up several components: an encoder to adjust the input data to a desired dimension, a **pos_encoder** to bestow the sequence with positional information, a core **transformer_encoder** comprising several layers to process the sequence, and a **decoder** to produce the final output. As data flows through the model during the forward pass, it undergoes a series of transformations: it's first projected to a higher dimension, then augmented with positional encodings, processed by the transformer layers, and finally, the last token's representation is harnessed to produce the output. An instance of this model is readily created and can be assigned to a computation device for further training or inference.



In [6]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

# Constructing the Transformer Model

The following code constructs the actual transformer-based model for time series prediction. The model is constructed to accept the following parameters.

* **input_dim**: The dimension of the input data, in this case we use only one input, the number of sunspots.
* **d_model**: The number of features in the transformer model's internal representations (also the size of embeddings). This controls how much a model can remember and process.
* **nhead**: The number of attention heads in the multi-head self-attention mechanism.
* **num_layers**: The number of transformer encoder layers.
dropout: The dropout probability.



In [7]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    # attention head = 4
    # encode layers = 2
    # input seq of size 1 (1 token)
    # embedding dim = 64
    # dropout = 0.2 --> every training step each neuron has 20% chance of not contributing (to the forward pass and  being updated during backward pass)
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model) #(input eg [128,20] with nn.Linear(20,50) --> output [128,50])
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel().to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


The Transformer architecture in PyTorch is governed by crucial configuration choices, among which **d_model**, **nhead**, and **num_layers** hold significant weight. The **d_model** denotes the dimensionality of the input embeddings and affects the model's capacity to learn intricate representations. While a more substantial **d_model** can bolster the richness of the model's understanding, it also amplifies the computational demand and can pose overfitting risks if not carefully chosen. Parallelly, the model's gradient flow and initialization are impacted by this choice, though the Transformer's normalization layers often moderate potential issues.

On the other hand, **nhead** reflects the count of heads in the multi-head attention mechanism. A higher number of heads grants the model the prowess to simultaneously focus on diverse segments of the input, enabling the capture of varied contextual nuances. However, there's a trade-off. Beyond a specific threshold, the computational overhead might outweigh the marginal performance gains. This parallel processing, provided by multiple attention heads, tends to offer more stable and varied gradient information, positively influencing the training dynamics.

Lastly, the **num_layers** parameter dictates the depth of the Transformer, determining the number of stacked encoder or decoder layers. A deeper model, as a result of increased layers, can discern more complex and hierarchical relationships in data. Still, there's a caveat: after a certain depth, potential performance enhancements may plateau, and the risk of overfitting might escalate. Training deeper models also comes with its set of challenges. Although residual connections and normalization in Transformers alleviate some concerns, a high layer count might necessitate techniques like gradient clipping or learning rate adjustments for stable training.

In essence, these parameters intricately balance model capacity, computational efficiency, and generalization capability. Their optimal settings often emerge from task-specific experimentation, the nature of the data, and available computational prowess.

## Training the Model


Training a transformer-based model adheres to many of the familiar paradigms and best practices that apply to other neural network architectures. Much like the models we've encountered before, a transformer-based model benefits from training in batches, which helps in both computational efficiency and generalization. Batched training ensures that the model updates its weights based on the average gradient over several data points, rather than being excessively influenced by any single instance. Additionally, the use of early stopping acts as a safeguard against overfitting. By monitoring the model's performance on a validation set and halting training when no significant improvement is observed over a set number of epochs, we ensure that the model generalizes well and doesn't just memorize the training data. The validation set, it remains an essential component in the training regimen, providing a proxy measure of the model's performance on unseen data and guiding hyperparameter tuning. Thus, while transformer architectures introduce novel mechanisms and complexities, the foundational principles of training deep learning models in PyTorch remain consistent.

In [13]:
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        print(x_batch.size())
        print('y:', y_batch.size())
        break
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")



/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Epoch 1/1000, Validation Loss: 0.0427
torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Epoch 2/1000, Validation Loss: 0.0427
torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Epoch 3/1000, Validation Loss: 0.0427
torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Epoch 4/1000, Validation Loss: 0.0427
torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Epoch 5/1000, Validation Loss: 0.0427
torch.Size([32, 10, 1])
y: torch.Size([32, 1])
Early stopping!


We can now evaluate the performance of this model.

In [31]:
# Evaluation
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.squeeze().tolist())

rmse = np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1, 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
print(f"Score (RMSE): {rmse:.4f}")

Score (RMSE): 15.4570


In [11]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(input.size(), input)
print(output)
print(output.size())

torch.Size([128, 20]) tensor([[-0.0574,  0.0051,  1.6921,  ..., -0.9250,  0.1460,  2.0136],
        [-0.3689, -0.4029, -0.2562,  ..., -2.6717, -0.2204, -0.9181],
        [ 0.9370,  1.3929,  1.2369,  ...,  0.8248,  0.2350,  0.6551],
        ...,
        [ 0.2851,  0.3624, -1.4330,  ..., -0.5621,  0.7308, -3.0019],
        [ 0.5037, -0.2625, -2.2336,  ...,  0.6224,  0.5143,  0.2612],
        [ 1.8728,  0.1446,  0.1530,  ...,  2.0829, -0.9029,  0.6513]])
tensor([[-0.6970,  0.2091, -0.1627,  ...,  0.5372, -0.7546, -0.9778],
        [-0.7493,  0.0980, -0.2785,  ...,  0.2001,  0.0895,  0.7037],
        [-0.7786, -0.2222,  0.0367,  ...,  0.7794, -0.0333, -0.4398],
        ...,
        [-0.0666,  0.5763, -0.5785,  ..., -0.0181,  0.4579,  0.3949],
        [-0.4420,  0.8439, -0.2057,  ..., -0.1597, -0.2559,  0.4057],
        [-0.3430,  0.0895,  0.4607,  ...,  0.6304, -1.0290, -0.2856]],
       grad_fn=<AddmmBackward0>)
torch.Size([128, 30])


# Decoder only model


In [7]:
# Positional Encoding for Transformer
class DecoderPositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(DecoderPositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


# Model definition using Transformer
class DecoderTransformerModel(nn.Module):
    # attention head = 4
    # encode layers = 2
    # input seq of size 1 (1 token)
    # embedding dim = 64
    # dropout = 0.2 --> every training step each neuron has 20% chance of not contributing (to the forward pass and  being updated during backward pass)
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(DecoderTransformerModel, self).__init__()

        # info
        self.model_type = "Transformer"
        self.d_model = d_model

        # define layers
        self.embedding = nn.Linear(input_dim, d_model) #(input eg [128,20] with nn.Linear(20,50) --> output [128,50])
        self.pos_encoder = DecoderPositionalEncoding(d_model, dropout)
        decoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=2048, dropout=dropout, activation="relu", layer_norm_eps=1e-05, batch_first=False, norm_first=False, bias=True, device=device)
        self.transformer_decoder = nn.TransformerEncoder(decoder_layers, num_layers,mask_check=True)
        self.linear = nn.Linear(d_model, 1)

    def forward(self, x, decoder_mask=None):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_decoder(x, decoder_mask)
        x = self.linear(x[:, -1, :]) #the output from the last token of the sequence is passed through a linear layer to produce the final model output.
        return x

    def get_decoder_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one element more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]

        return mask

decoder_model = DecoderTransformerModel().to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [11]:
# Train the decoder model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(decoder_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    decoder_model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        # x_input = x_batch[:,:-1]
        # x_expected = x_batch[:,1:]

        # Get mask to mask out the next words
        sequence_length = x_batch.size(0)
        decoder_mask = decoder_model.get_decoder_mask(sequence_length).to(device)

        optimizer.zero_grad()
        outputs = decoder_model(x_batch,decoder_mask)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    decoder_model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = decoder_model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/1000, Validation Loss: 0.0688
Epoch 2/1000, Validation Loss: 0.0724
Epoch 3/1000, Validation Loss: 0.0359
Epoch 4/1000, Validation Loss: 0.0432
Epoch 5/1000, Validation Loss: 0.0587
Epoch 6/1000, Validation Loss: 0.0364
Epoch 7/1000, Validation Loss: 0.0442
Early stopping!


In [12]:
# Evaluation of decodr model
decoder_model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = decoder_model(x_batch)
        predictions.extend(outputs.squeeze().tolist())

rmse = np.sqrt(np.mean((scaler.inverse_transform(np.array(predictions).reshape(-1, 1)) - scaler.inverse_transform(y_test.numpy().reshape(-1, 1)))**2))
print(f"Score (RMSE): {rmse:.4f}")

Score (RMSE): 17.7250


#Decode Only with